# PH Download Speeds

This data project's overall goal is to identify cities or municipalities in the Philippines with a good internet connection and potentially be a location for a work-from-home setup. 

There are two primary steps: the data prep and the visualization step. This notebook will focus on the data prep. 

Key steps in the data preprocessing:
1. Join the data from [Speedtest by Ookla](https://registry.opendata.aws/speedtest-global-performance/) and join it with the Philippine boundaries data from [HDX](https://data.humdata.org/dataset/philippines-administrative-levels-0-to-3) 
2. Aggregate the data to the city/town level and add speed brackets and other data 
3. Export the data to a shp file that is to be visualized in Tableau 

The data preprocessing stage is heavily lifted from [katiejolly-ookla](https://github.com/teamookla/ookla-open-data/blob/master/tutorials/aggregate_by_county_py.ipynb).   

In [2]:
%matplotlib inline

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text
import os

In [3]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

tile_url = get_tile_url("mobile", 2021, 1)
tile_url

'https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance/type%3Dmobile/year%3D2021/quarter%3D1/2021-01-01_performance_mobile_tiles.zip'

## Downloading and joining the necessary files

You can directly read the files from the website. However, I chose to download the files to my local machine. 

In [4]:
curd = os.getcwd()
tiles = gp.read_file(curd+'/downloads/2021-01-01_performance_mobile_tiles.zip')
fixed = gp.read_file(curd+'/downloads/2021-01-01_performance_fixed_tiles.zip')
counties = gp.read_file(curd+'/downloads/admin3.zip')

In [5]:
tiles_in_cities = gp.sjoin(counties, tiles, how='left', op='intersects')
fixed_in_cities = gp.sjoin(counties, fixed, how='left', op='intersects')

## Aggregating and adding speed brackets and other data

For the visualizaion step, I had to have the data aggregated to the cities/town level due to file size consideration. Also, I had to add labels, speed brackets, convert the unit of measurement etc. 


In [6]:
tiles_in_cities['con_type'] = 'mobile'
fixed_in_cities['con_type'] = 'fixed'
ph_net_speeds = pd.concat([tiles_in_cities,fixed_in_cities])

In [7]:
ph_net_speeds['avg_d_mbps'] = ph_net_speeds['avg_d_kbps'] / 1000
ph_net_speeds['avg_u_mbps'] = ph_net_speeds['avg_u_kbps'] / 1000

In [8]:
labels = ["0 to 5 Mbps", "5 to 10 Mbps", "10 to 15 Mbps", "15 to 25 Mbps", "25 to 200 Mbps"]

ph_net_speeds['group'] = pd.cut(
    ph_net_speeds.avg_d_mbps, 
    (0, 5, 10, 15, 25, 200), 
    right=False, 
    labels = labels
)

# Subset the data for aggregation 
ph_contri = ph_net_speeds[['ADM3_PCODE','ADM3_EN','con_type','group','tests']]

In [19]:
# aggregate the data at the speed bracket level. This will be used to calculate the % contribution of tests 
z_group = ph_contri
z_group = z_group.groupby(['ADM3_PCODE','con_type','group'])\
.agg({"tests" : "sum"})[["tests"]]
z_group = z_group.reset_index()

# aggregate the data at the connection type level
p_cont_type = ph_contri.groupby(['ADM3_PCODE','con_type'])\
.agg({"tests" : "sum"})[["tests"]]
# https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe
p_con_type = p_cont_type.add_suffix('_total').reset_index()

# combine the aggregations to 
result = pd.merge(left=z_group, right=p_con_type,\
                  left_on=['ADM3_PCODE','con_type'], right_on=['ADM3_PCODE','con_type'])

# calculate the % contribution and pivot the data
result['perc_cont'] = result.tests/result.tests_total*100
result = pd.pivot_table(result, values = 'perc_cont', index=['ADM3_PCODE', 'con_type'], columns = 'group')
result = result.add_suffix('_contri').reset_index()
speed_distro = result

In [20]:
# get the totals for tests, avg speed, and device count  
county_stats = (
    ph_net_speeds.groupby(["ADM3_EN","ADM3_PCODE","con_type"])
    .apply(
        lambda x: pd.Series(
            {"avg_d_mbps_wt": np.average(x["avg_d_mbps"], weights=x["tests"])}
        )
    )
    .reset_index()
    .merge(
        ph_net_speeds.groupby(["ADM3_EN","ADM3_PCODE","con_type"])
        .agg(tests=("tests", "sum"),devices=("devices","sum"))
        .reset_index(),
        on=["ADM3_EN","ADM3_PCODE","con_type"],
    )
)

## Combine all the aggregations to one dataframe and export to a shp file

In [21]:
to_tab = pd.merge(left=county_stats, right=speed_distro,\
                  left_on=['ADM3_PCODE','con_type'], right_on=['ADM3_PCODE','con_type'])
to_tab = pd.merge(left=counties, right=to_tab, left_on=['ADM3_PCODE'], right_on=['ADM3_PCODE'])
to_tab.to_file(curd+'/downloads/ph_net_speeds.shp')

<ipython-input-21-646b40f460fe>:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  to_tab.to_file(curd+'/downloads/ph_net_speeds.shp')
